In [30]:
import json, os
import copy
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

from langchain_community.vectorstores.utils import filter_complex_metadata
import pprint
from dotenv import find_dotenv, load_dotenv
from langchain_chroma import Chroma
from rich.console import Console

console = Console()
load_dotenv()

embeddings = OpenAIEmbeddings()

file = "TMCB_43_2256640.pdf.json"
persist_directory = "./data/db/chroma/"
data_json_directory = "./pdf_output/"

# get all the json from json file

file = data_json_directory + file
# console.print(f"File: {file}")

# import json data
with open(file) as f:
    data = json.load(f)
    # console.print(data)

# console.print(f"Length: {len(data)}")
# console.print(data[0])
metadata = dict(data[0]["metadata"])
el_type = data[0]["type"]
page_number = metadata["page_number"]
doc_id = data[0]["element_id"]
content = data[0]["text"]
# console.print(f"[green]Content: {content}[/]")
# console.print(f"[yellow]Metadata: {metadata}[/]")

In [27]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
from dotenv import load_dotenv

app_dir = os.path.join(os.getcwd(), "app")
load_dotenv(os.path.join(app_dir, ".env"))

embedding_function = OpenAIEmbeddings()

docs = [
    Document(
        page_content=content,
        metadata={
            "type": el_type,
            "page_number": page_number,
            "doc_id": doc_id,
            "filename": file,
        },
    ),
]


In [3]:
from langchain.prompts.prompt import PromptTemplate

rephrase_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
REPHRASE_TEMPLATE = PromptTemplate.from_template(rephrase_template)

In [4]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

rephrase_chain = REPHRASE_TEMPLATE | ChatOpenAI(temperature=0) | StrOutputParser()

In [ ]:
rephrase_chain.invoke(
    {
        "question": "No, really?",
        "chat_history": [
            HumanMessage(content="What does the dog like to eat?"),
            AIMessage(content="Thuna!"),
        ],
    }
)

In [6]:
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [7]:
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | ANSWER_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

In [8]:
final_chain = rephrase_chain | retrieval_chain

In [ ]:
final_chain.invoke(
    {
        "question": "No, really?",
        "chat_history": [
            HumanMessage(content="What does the dog like to eat?"),
            AIMessage(content="Thuna!"),
        ],
    }
)

### Chat with returning documents

In [10]:
retrieved_documents = {"docs": retriever, "question": RunnablePassthrough()}
final_inputs = {
    "context": lambda x: "\n".join(doc.page_content for doc in x["docs"]),
    "question": lambda x: x["question"],
}
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI() | StrOutputParser(),
    "docs": lambda x: x["docs"],
}

final_chain = rephrase_chain | retrieved_documents | answer

In [ ]:
result = final_chain.invoke(
    {
        "question": "No, really?",
        "chat_history": [
            HumanMessage(content="What does the dog like to eat?"),
            AIMessage(content="Thuna!"),
        ],
    }
)
print(result)

In [ ]:
result["answer"]

In [ ]:
result["docs"]